In [1]:
import os
import random
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [ ]:
# Neural Network Class

# Fully connected neural network with number of layers, neurons, input size, output size and activation function as parameters

class NeuralNetwork(nn.Module):
    def __init__(self, num_layers, num_neurons, input_size, output_size, activation_function):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.activation_function = activation_function

        # Input layer
        self.layers.append(nn.Linear(input_size, num_neurons))

        # Hidden layers
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(num_neurons, num_neurons))

        # Output layer
        self.layers.append(nn.Linear(num_neurons, output_size))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation_function(layer(x))
        x = self.layers[-1](x)
        return x

## define the activation functions: tanh() and sin()
def tanh(x):
    return torch.tanh(x)

def sin(x):
    return torch.sin(x)

# input size = 2, (x, t)


In [ ]:
# initial condition 
def ic_func(x):
    return np.cos(x[:, 0:1]) + 0.1 * np.sin(2 * x[:, 0:1])


In [ ]:
# boundary condition

## periodic boundary condition


In [ ]:
# Loss Function construction

## Residual loss function

def Residual_NN(x, u, t, u_t, u_xx):
    # Compute the residual of the PDE
    f = u_t + u_xx - np.sin(t) * np.cos(x)

    dy_dx = torch.autograd.grad(y, x, torch.ones_like(y), create_graph=True)[0]
    return f

## Boundary loss function
def Boundary_NN(x, t, u):
    # Compute the boundary condition loss
    u_bc = torch.zeros_like(u)
    return u - u_bc

## Initial loss function
def Initial_NN(x, u):
    # Compute the initial condition loss
    u_ic = ic_func(x)
    return u - u_ic

## training loss function
def train_loss(data, model):
    x = x.requires_grad_()
    t = t.requires_grad_()
    u = u.requires_grad_()
    u_t = u_t.requires_grad_()
    u_xx = u_xx.requires_grad_()


    # Compute the total loss
    res_loss = Residual_NN(x, u, t, u_t, u_xx)
    bc_loss = Boundary_NN(x, t, u)
    ic_loss = Initial_NN(x, u)
    total_loss = res_loss + bc_loss + ic_loss
    return total_loss, res_loss,  ic_loss

import torch

# === Helper: Compute Derivatives for KS Equation ===
def compute_ks_derivatives(model, x, t):
    """
    Given a model that predicts u(x,t), compute:
      u, u_t, u_x, u_xx, u_xxx, and u_xxxx 
    using automatic differentiation.

    Inputs:
      model : the neural network model, taking (x, t)
      x, t  : tensors with requires_grad=True
  
    Returns:
      u, u_t, u_x, u_xx, u_xxx, u_xxxx
    """
    # Make sure x and t require gradients
    x = x.requires_grad_()
    t = t.requires_grad_()

    # Forward pass: predict u
    u = model(x, t)

    # First derivative with respect to time: u_t
    u_t = torch.autograd.grad(
        u, t,
        grad_outputs=torch.ones_like(u),
        retain_graph=True,
        create_graph=True
    )[0]

    # First derivative with respect to space: u_x
    u_x = torch.autograd.grad(
        u, x,
        grad_outputs=torch.ones_like(u),
        retain_graph=True,
        create_graph=True
    )[0]

    # Second derivative with respect to space: u_xx
    u_xx = torch.autograd.grad(
        u_x, x,
        grad_outputs=torch.ones_like(u_x),
        retain_graph=True,
        create_graph=True
    )[0]

    # Third derivative with respect to space: u_xxx
    u_xxx = torch.autograd.grad(
        u_xx, x,
        grad_outputs=torch.ones_like(u_xx),
        retain_graph=True,
        create_graph=True
    )[0]

    # Fourth derivative with respect to space: u_xxxx
    u_xxxx = torch.autograd.grad(
        u_xxx, x,
        grad_outputs=torch.ones_like(u_xxx),
        retain_graph=True,
        create_graph=True
    )[0]

    return u, u_t, u_x, u_xx, u_xxx, u_xxxx


# === Residual Loss Function for KS Equation ===
def residual_loss(model, x_f, t_f, nu):
    """
    Computes the residual loss for the KS equation:
       u_t + u*u_x + u_xx + nu*u_xxxx = 0
  
    Inputs:
      model: neural network that predicts u(x,t)
      x_f, t_f: collocation (interior) points where the PDE is enforced
      nu: parameter nu in the equation

    Returns:
      loss_res: a scalar tensor representing the Mean Squared Error (MSE)
                of the PDE residual.
    """
    # Compute u and necessary derivatives at collocation points:
    u, u_t, u_x, u_xx, _, u_xxxx = compute_ks_derivatives(model, x_f, t_f)
    
    # Compute the PDE residual f:
    # f = u_t + u*u_x + u_xx + nu*u_xxxx
    f = u_t + u * u_x + u_xx + nu * u_xxxx

    # Mean Squared Error of the residual (forcing f=0):
    loss_res = torch.mean(f**2)
    return loss_res


# === Boundary Loss Function for Periodic BC ===
def boundary_loss(model, x_left, t_left, x_right, t_right):
    """
    Computes the boundary loss for enforcing periodic boundary conditions.
    For a periodic domain [a, b] we enforce u(a,t) = u(b,t).

    Inputs:
      model: neural network that predicts u(x,t)
      x_left, t_left: points on the left boundary (e.g., x=a)
      x_right, t_right: corresponding points on the right boundary (e.g., x=b)

    Returns:
      loss_bc: the Mean Squared Error (MSE) between u(x_left,t_left) and u(x_right,t_right).
    """
    u_left = model(x_left, t_left)
    u_right = model(x_right, t_right)

    loss_bc = torch.mean((u_left - u_right)**2)
    return loss_bc


# === Initial Loss Function ===
def initial_loss(model, x_ic, t_ic, u_ic_target):
    """
    Computes the loss for the initial condition:
        u(x, t=0) = u_ic_target(x)

    Inputs:
      model: neural network that predicts u(x,t)
      x_ic, t_ic: the initial condition points (typically t_ic is zero)
      u_ic_target: true initial values obtained from your ic function

    Returns:
      loss_ic: Mean Squared Error (MSE) between network prediction and initial condition target.
    """
    u_ic_pred = model(x_ic, t_ic)
    loss_ic = torch.mean((u_ic_pred - u_ic_target)**2)
    return loss_ic


# === Combined Training Loss Function ===
def train_loss(model, data, nu):
    """
    Computes the total loss by combining:
      - the residual loss (enforcing the KS PDE)
      - the boundary loss (enforcing periodic BC)
      - the initial loss (enforcing the IC)
      
    The `data` dictionary is assumed to have the following keys:
      - 'collocation': tuple (x_f, t_f)
      - 'boundary': tuple (x_left, t_left, x_right, t_right)
      - 'initial': tuple (x_ic, t_ic, u_ic_target)
      
    nu is the KS equation parameter.

    Returns:
      total_loss: the sum of the three loss terms.
      loss_r, loss_b, loss_i: individual loss components.
    """
    # Unpack the data dictionary
    x_f, t_f = data['collocation']
    x_left, t_left, x_right, t_right = data['boundary']
    x_ic, t_ic, u_ic_target = data['initial']

    # Compute each loss term:
    loss_r = residual_loss(model, x_f, t_f, nu)
    loss_b = boundary_loss(model, x_left, t_left, x_right, t_right)
    loss_i = initial_loss(model, x_ic, t_ic, u_ic_target)

    # Combine the losses (you can also weight these if needed)
    total_loss = loss_r + loss_b + loss_i
    return total_loss, loss_r, loss_b, loss_i


In [ ]:
# training process
epochs = 5000

loss_history = []
ode_loss_history = []
initial_loss_history = []

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = NeuralNetwork(num_layers=4, num_neurons=50, input_size=1, output_size=1, activation_function=tanh)
print(model)

x_train = torch.linspace(-2, 2, 100).view(-1, 1)  # Training points

for epoch in range(epochs):
    optimizer.zero_grad()
    total_loss, ode_loss, initial_loss = train_loss(model, x_train)
    total_loss.backward()
    optimizer.step()

    
    loss_history.append(total_loss.item())
    ode_loss_history.append(ode_loss.item())
    initial_loss_history.append(initial_loss.item())

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.item():.6f}")


